In [1]:
import pandas as pd
import numpy as np
from bert_serving.client import BertClient
from cnsenti import Sentiment

### 可读性、语调 保存为text_norm

In [2]:
# 读取
data_clean_2018_2020_path ='F:\\zzqaq\\data\\data_clean_2018_2020.csv'
data_clean_2018_2020 = pd.read_csv(data_clean_2018_2020_path,encoding="gb18030",converters={'code':str,'year':str,'board':str})
print(data_clean_2018_2020.shape)
data_clean_2018_2020[:2]

(7781, 4)


,code,year,text,board
0,000004,2018,讨论与分析一概述年度，公司实现主营业务收入万元，比上年同期增加，实现归属于上市公司股东的净利...,00
1,000004,2020,讨论与分析一概述年是公司业务转型后的第一年，年，公司筹划发行股份收购智游网安股权的重大资产重...,00


In [3]:
# from cnsenti import Sentiment
senti = Sentiment()
# 例子
test_text = "在本报告期内，本司按如下的框架发展主营业务交通清洁能源水资源基础设施经营；"
result = senti.sentiment_count(test_text)
print('sentiment_count',result)

sentiment_count {'words': 19, 'sentences': 1, 'pos': 3, 'neg': 0}


In [4]:
# 调用金融词典
path_pos = 'F:\\zzqaq\\text_data_processs\\中文金融词典\\formal_pos.txt'
path_neg = 'F:\\zzqaq\\text_data_processs\\中文金融词典\\formal_neg.txt'

senti = Sentiment(pos = path_pos,  #正面词典txt文件相对路径
                  neg = path_neg,  #负面词典txt文件相对路径
                  encoding='utf-8')      #两txt均为utf-8编码

dicto={}
for i in range(data_clean_2018_2020['text'].shape[0]):
    try:
        text = data_clean_2018_2020['text'][i]
        result = senti.sentiment_count(text)
        dicto[str(i)]=result
        if i % 500 == 0 and i != 0: print("Printing number:", i)
    except:
        print("Error occurred at iteration:", i)
df_dict = pd.DataFrame.from_dict(dicto, orient='index')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\asus\AppData\Local\Temp\jieba.cache
Loading model cost 0.636 seconds.
Prefix dict has been built successfully.


Printing number: 500
Printing number: 1000
Printing number: 1500
Printing number: 2000
Printing number: 2500
Printing number: 3000
Printing number: 3500
Printing number: 4000
Printing number: 4500
Printing number: 5000
Printing number: 5500
Printing number: 6000
Printing number: 6500
Printing number: 7000
Printing number: 7500


In [8]:
text_norm = corpus_1820.loc[:, ['code', 'year']].copy()

# 赋值
text_norm['word_count'] = list(df_dict['words'])
text_norm['sentence_count'] = list(df_dict['sentences'])
text_norm['readability'] = text_norm['word_count']/text_norm['sentence_count']

text_norm['pos_count'] = list(df_dict['pos'])
text_norm['neg_count'] = list(df_dict['neg'])
text_norm['tone'] = (text_norm['pos_count']-text_norm['neg_count'])/(text_norm['pos_count']+text_norm['neg_count'])
text_norm

,code,year,word_count,sentence_count,readability,pos_count,neg_count,tone
0,000004,2018,3146,59,53.322034,286,69,0.611268
1,000004,2020,7359,162,45.425926,801,206,0.590864
2,000005,2018,6397,136,47.036765,644,108,0.712766
3,000005,2019,5796,129,44.930233,608,111,0.691238
4,000005,2020,4798,112,42.839286,474,107,0.631670
...,...,...,...,...,...,...,...,...
7776,900956,2018,3142,105,29.923810,479,67,0.754579
7777,900956,2019,3463,110,31.481818,508,81,0.724958
7778,900957,2018,2768,88,31.454545,306,91,0.541562
7779,900957,2019,3213,106,30.311321,340,118,0.484716


In [9]:
# 保存
text_norm.to_csv('F:\\zzqaq\\data\\text_norm.csv',encoding="gb18030",index = False)

In [10]:
# 读取
text_norm ='F:\\zzqaq\\data\\text_norm.csv'
text_norm = pd.read_csv(text_norm,encoding="gb18030",converters={'code':str,'year':str,'board':str})
text_norm.head(2)

,code,year,word_count,sentence_count,readability,pos_count,neg_count,tone
0,000004,2018,3146,59,53.322034,286,69,0.611268
1,000004,2020,7359,162,45.425926,801,206,0.590864
